In [1]:
import jax
from typing import Any, Callable, Sequence, Optional
from jax import lax, random, numpy as jnp
import flax
from flax.core import freeze, unfreeze
from flax import linen as nn
import optax
from sklearn.metrics import accuracy_score

Loading Dataset

In [2]:
from ast import literal_eval
from collections import Counter
from datetime import datetime
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score, classification_report
from sherlock.deploy.model import SherlockModel

start = datetime.now()

X_train = pd.read_parquet('../data/data/processed/train.parquet')
y_train = pd.read_parquet('../data/data/raw/train_labels.parquet').values.flatten()
y_train = np.array([x.lower() for x in y_train])

X_validation = pd.read_parquet('../data/data/processed/validation.parquet')
y_validation = pd.read_parquet('../data/data/raw/val_labels.parquet').values.flatten()
y_validation = np.array([x.lower() for x in y_validation])

X_test = pd.read_parquet('../data/data/processed/test.parquet')
y_test = pd.read_parquet('../data/data/raw/test_labels.parquet').values.flatten()
y_test = np.array([x.lower() for x in y_test])

print(f'Load data process took {datetime.now() - start} seconds.')

2022-05-09 10:07:23.776520: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory


Load data process took 0:00:17.063918 seconds.


In [3]:
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

from sherlock.deploy import helpers

num_classes = len(set(y_train))

encoder = LabelEncoder()
encoder.fit(y_train)

feature_cols = helpers.categorize_features()

X_train_char = X_train[feature_cols["char"]]
X_train_word = X_train[feature_cols["word"]]
X_train_par = X_train[feature_cols["par"]]
X_train_rest = X_train[feature_cols["rest"]]

X_val_char = X_validation[feature_cols["char"]]
X_val_word = X_validation[feature_cols["word"]]
X_val_par = X_validation[feature_cols["par"]]
X_val_rest = X_validation[feature_cols["rest"]]

y_train_int = encoder.transform(y_train)   #(412059,)
y_train_cat = tf.keras.utils.to_categorical(y_train_int) #(412059,78)
y_val_int = encoder.transform(y_validation)
y_val_cat = tf.keras.utils.to_categorical(y_val_int)

/mnt/d/GitCode/sherlock-project/sherlock/deploy/helpers.py:18: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  feature_cols_dict[feature_set] = pd.read_csv(
/mnt/d/GitCode/sherlock-project/sherlock/deploy/helpers.py:18: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  feature_cols_dict[feature_set] = pd.read_csv(
/mnt/d/GitCode/sherlock-project/sherlock/deploy/helpers.py:18: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  feature_cols_dict[feature_set] = pd.read_csv(
/mnt/d/GitCode/sherlock-project/sherlock/deploy/helpers.py:18: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  fe

In [4]:
# =================================================================================================
#  _build_char_submodel with flax - line 241 from model.py

'''
https://flax.readthedocs.io/en/latest/flax.linen.html#module
class Module(nn.Module):
  features: Tuple[int] = (960, 200, 400, 27)

  def setup(self):
    self.dense1 = Dense(self.features[0])
    self.dense2 = Dense(self.features[1])
    self.dense3 = Dense(self.features[2])
    self.dense4 = Dense(self.features[3])
    
  def __call__(self, x):
    combined2 = self.dense2(nn.relu(self.dense1(x)))
    combined3 = self.dense3(nn.relu(combined2))
    combined4 = self.dense3(nn.relu(combined3))
    
    return combined4
'''

#https://flax.readthedocs.io/en/latest/notebooks/flax_basics.html
'''char_model = nn.Dense(features=300)
key1, key2 = random.split(random.PRNGKey(0))
x = random.normal(key1, (960,)) # Dummy input
params = char_model.init(key2, x) # Initialization call
print(jax.tree_map(lambda x: x.shape, params)) # Checking output shapes


word_model = nn.Dense(features=200)
key1, key2 = random.split(random.PRNGKey(0))
x = random.normal(key1, (200,)) # Dummy input
params = word_model.init(key2, x) # Initialization call
print(jax.tree_map(lambda x: x.shape, params))


par_model = nn.Dense(features=400)
key1, key2 = random.split(random.PRNGKey(0))
x = random.normal(key1, (400,)) # Dummy input
params = par_model.init(key2, x) # Initialization call
print(jax.tree_map(lambda x: x.shape, params))

rest_model = nn.Dense(features=27)
key1, key2 = random.split(random.PRNGKey(0))
x = random.normal(key1, (27,)) # Dummy input
params = rest_model.init(key2, x) # Initialization call
print(jax.tree_map(lambda x: x.shape, params))'''

#main - concat all todo - main.py line65

# <=================================================================================================
'''
model = nn.Dense(features=1)

#Model parameters & initialization
key1, key2 = random.split(random.PRNGKey(0))
x = random.normal(key1, (3,)) # Dummy input
params = model.init(key2, x) # Initialization call
jax.tree_map(lambda x: x.shape, params) # Checking output shapes


char_model_input, char_model = _build_char_submodel(X_train_char.shape[1]) #960
word_model_input, word_model = self._build_word_submodel(X_train_word.shape[1]) #200
par_model_input, par_model = self._build_par_submodel(X_train_par.shape[1])     #400
rest_model_input, rest_model = self._build_rest_submodel(X_train_rest.shape[1]) #27
'''


#todo 2 - merge model and build main network
# model.py from line 64


'\nmodel = nn.Dense(features=1)\n\n#Model parameters & initialization\nkey1, key2 = random.split(random.PRNGKey(0))\nx = random.normal(key1, (3,)) # Dummy input\nparams = model.init(key2, x) # Initialization call\njax.tree_map(lambda x: x.shape, params) # Checking output shapes\n\n\nchar_model_input, char_model = _build_char_submodel(X_train_char.shape[1]) #960\nword_model_input, word_model = self._build_word_submodel(X_train_word.shape[1]) #200\npar_model_input, par_model = self._build_par_submodel(X_train_par.shape[1])     #400\nrest_model_input, rest_model = self._build_rest_submodel(X_train_rest.shape[1]) #27\n'

In [5]:
print(X_train.shape, y_train.shape) #df and np ndarray

(412059, 1588) (412059,)


960 single char level model training

Dense_0: {
    bias: (300,),
    kernel: (960, 300),
},
Dense_1: {
    bias: (300,),
    kernel: (300, 300),
},
Dense_2: {
    bias: (78,),
    kernel: (300, 78),
    
1 layer + 2*300 layers + 78 output layer


In [6]:
#try 960 model only

class MLP(nn.Module):
  features: Sequence[int]

  @nn.compact
  def __call__(self, x):
    for feat in self.features[:-1]:
      x = nn.relu(nn.Dense(feat)(x))
    x = nn.Dense(self.features[-1])(x) # ?
    return x

model = MLP([300, 300, 78]) # do we need 78 layer for model concat..?
#dummy_input = jnp.ones((412059, 960))
#params = model.init(jax.random.PRNGKey(0), dummy_input)


#==================================
rng = jax.random.PRNGKey(0)
rng1, rng2 = jax.random.split(rng)
params = model.init(rng2, jax.random.normal(rng1, (960,)))
print(params)


## predict
print(X_train_char.shape) #(412059, 960)
y = model.apply(params, X_train_char)
print(y.shape) #(412059, 78)

jax.tree_map(lambda dummy_input: dummy_input.shape, params) # Checking output shapes


FrozenDict({
    params: {
        Dense_0: {
            kernel: DeviceArray([[ 0.06141227,  0.00766806, -0.05698649, ...,  0.01272237,
                           0.01838974,  0.02176958],
                         [-0.02200374, -0.07047574, -0.00778548, ..., -0.03661647,
                          -0.01194724,  0.01908318],
                         [-0.00935888,  0.00299105, -0.0068249 , ..., -0.05483543,
                           0.03303373, -0.02801319],
                         ...,
                         [ 0.01294531, -0.00636607, -0.04631356, ...,  0.03514464,
                          -0.00600298, -0.03478616],
                         [-0.04506306,  0.03920445, -0.05522922, ..., -0.05057716,
                          -0.01210736, -0.00505882],
                         [ 0.05229904, -0.02613135, -0.0082468 , ..., -0.04306015,
                           0.00207464, -0.04712147]], dtype=float32),
            bias: DeviceArray([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

FrozenDict({
    params: {
        Dense_0: {
            bias: (300,),
            kernel: (960, 300),
        },
        Dense_1: {
            bias: (300,),
            kernel: (300, 300),
        },
        Dense_2: {
            bias: (78,),
            kernel: (300, 78),
        },
    },
})

In [7]:
#print(params)
print(params["params"]["Dense_0"]["kernel"])


print(params["params"]["Dense_0"]["kernel"].shape)

[[ 0.06141227  0.00766806 -0.05698649 ...  0.01272237  0.01838974
   0.02176958]
 [-0.02200374 -0.07047574 -0.00778548 ... -0.03661647 -0.01194724
   0.01908318]
 [-0.00935888  0.00299105 -0.0068249  ... -0.05483543  0.03303373
  -0.02801319]
 ...
 [ 0.01294531 -0.00636607 -0.04631356 ...  0.03514464 -0.00600298
  -0.03478616]
 [-0.04506306  0.03920445 -0.05522922 ... -0.05057716 -0.01210736
  -0.00505882]
 [ 0.05229904 -0.02613135 -0.0082468  ... -0.04306015  0.00207464
  -0.04712147]]
(960, 300)


### Optimizing with Optax

Flax used to use its own `flax.optim` package for optimization, but with
[FLIP #1009](https://github.com/google/flax/blob/main/docs/flip/1009-optimizer-api.md)
this was deprecated in favor of
[Optax](https://github.com/deepmind/optax).

Basic usage of Optax is straightforward:

1.   Choose an optimization method (e.g. `optax.sgd`).
2.   Create optimizer state from parameters.
3.   Compute the gradients of your loss with `jax.value_and_grad()`.
4.   At every iteration, call the Optax `update` function to update the internal
     optimizer state and create an update to the parameters. Then add the update
     to the parameters with Optax's `apply_updates` method.

Note that Optax can do a lot more: it's designed for composing simple gradient
transformations into more complex transformations that allows to implement a
wide range of optimizers. There is also support for changing optimizer
hyperparameters over time ("schedules"), applying different updates to different
parts of the parameter tree ("masking") and much more. For details please refer
to the
[official documentation](https://optax.readthedocs.io/en/latest/).


In [8]:
#print(params)

learning_rate = 0.0001
n_training_steps = 100

# Define an MSE loss function.
def make_mse_func(x_batched, y_batched):
  def mse(params):    
    # Define the squared loss for a single (x, y) pair.
    def squared_error(x, y):      
      pred = model.apply(params, x)
      print("y and y pred")
      print(y[0])
      print(pred[0])
    
      return jnp.inner(y-pred, y-pred) / 2.0
    
    print("x and y batched")
    print(x_batched.shape)
    print(y_batched.shape)
    
    
    # Vectorise the squared error and compute the average of the loss.
    return jnp.mean(jax.vmap(squared_error)(x_batched, y_batched), axis=0)
  return jax.jit(mse)  # `jit` the result.

ver1. Using dummy sample

In [9]:
# Set problem dimensions.
nsamples = 20
xdim = 960
ydim = 78

# Generate random ground truth w and b.
w = jax.random.normal(rng1, (xdim, ydim))
b = jax.random.normal(rng2, (ydim,))

# Generate samples with additional noise.
ksample, knoise = jax.random.split(rng1)
x_samples = jax.random.normal(ksample, (nsamples, xdim))
y_samples = jnp.dot(x_samples, w) + b
y_samples += 0.1 * jax.random.normal(knoise, (nsamples, ydim))


#================================================

print(y_samples[0][0])

# Instantiate the sampled loss.
loss = make_mse_func(x_samples, y_samples)
optimizer = optax.adam(learning_rate=1e-2)

# Create optimiser state.
opt_state = optimizer.init(params)
# Compute the gradient of the loss function.
loss_grad_fn = jax.value_and_grad(loss)

10.051781


In [10]:
# Minimise the loss.
for step in range(n_training_steps):
    # Compute gradient of the loss.
    loss_val, grads = loss_grad_fn(params)
    # Update the optimiser state, create an update to the params.
    updates, opt_state = optimizer.update(grads, opt_state)
    # Update the parameters.
    params = optax.apply_updates(params, updates)
    if step%10 == 0: print(f'Loss[{step}] = {loss_val}')

x and y batched
(20, 960)
(20, 78)
y and y pred
Traced<ShapedArray(float32[])>with<BatchTrace(level=3/1)> with
  val = Traced<ShapedArray(float32[20])>with<DynamicJaxprTrace(level=0/1)>
  batch_dim = 0
Traced<ShapedArray(float32[])>with<BatchTrace(level=3/1)> with
  val = Traced<ShapedArray(float32[20])>with<JVPTrace(level=2/1)> with
    primal = Traced<ShapedArray(float32[20])>with<DynamicJaxprTrace(level=0/1)>
    tangent = Traced<ShapedArray(float32[20])>with<JaxprTrace(level=1/1)> with
      pval = (ShapedArray(float32[20]), *)
      recipe = JaxprEqnRecipe(eqn_id=<object object at 0x7fe696069c70>, invars=(Traced<ShapedArray(float32[20,78]):JaxprTrace(level=1/1)>, Traced<ShapedArray(int32[1]):JaxprTrace(level=1/1)>), outvars=[<weakref at 0x7fe68ec11400; to 'JaxprTracer' at 0x7fe68ec113b0>], primitive=gather, params={'dimension_numbers': GatherDimensionNumbers(offset_dims=(0,), collapsed_slice_dims=(1,), start_index_map=(1,)), 'slice_sizes': (20, 1), 'unique_indices': True, 'indices

ver2. Using training data:  df -> jnp

http://bl.ocks.org/miguelusque/raw/f44a8e729896a96d0a3e4b07b5176af4/#numpy-jax

In [11]:
npy = pd.DataFrame(X_train_char).to_numpy()
dst_x = jnp.array(npy)
dst_y = jnp.array(y_train_cat)

# Instantiate the sampled loss.
loss = make_mse_func(dst_x, dst_y)

optimizer = optax.adam(learning_rate=1e-2)

# Create optimiser state.
opt_state = optimizer.init(params)
# Compute the gradient of the loss function.
loss_grad_fn = jax.value_and_grad(loss)


ver2. Execute training

In [12]:
# Minimise the loss.
start = datetime.now()

for step in range(100):
    # Compute gradient of the loss.
    loss_val, grads = loss_grad_fn(params)
    # Update the optimiser state, create an update to the params.
    updates, opt_state = optimizer.update(grads, opt_state)
    # Update the parameters.
    params = optax.apply_updates(params, updates)
    if step%100 == 0: 
        print(f'Loss[{step}] = {loss_val}')
        
print(f'Load data process took {datetime.now() - start} seconds.')

x and y batched
(412059, 960)
(412059, 78)
y and y pred
Traced<ShapedArray(float32[])>with<BatchTrace(level=3/1)> with
  val = Traced<ShapedArray(float32[412059])>with<DynamicJaxprTrace(level=0/1)>
  batch_dim = 0
Traced<ShapedArray(float32[])>with<BatchTrace(level=3/1)> with
  val = Traced<ShapedArray(float32[412059])>with<JVPTrace(level=2/1)> with
    primal = Traced<ShapedArray(float32[412059])>with<DynamicJaxprTrace(level=0/1)>
    tangent = Traced<ShapedArray(float32[412059])>with<JaxprTrace(level=1/1)> with
      pval = (ShapedArray(float32[412059]), *)
      recipe = JaxprEqnRecipe(eqn_id=<object object at 0x7fe6745d76a0>, invars=(Traced<ShapedArray(float32[412059,78]):JaxprTrace(level=1/1)>, Traced<ShapedArray(int32[1]):JaxprTrace(level=1/1)>), outvars=[<weakref at 0x7fe6741ddae0; to 'JaxprTracer' at 0x7fe6741dda90>], primitive=gather, params={'dimension_numbers': GatherDimensionNumbers(offset_dims=(0,), collapsed_slice_dims=(1,), start_index_map=(1,)), 'slice_sizes': (412059, 

In [13]:
#check param
print(params["params"]["Dense_0"]["kernel"])
print(params["params"]["Dense_0"]["kernel"].shape)

[[ 0.12023748 -0.10125136  0.03266515 ...  0.02326848 -0.00838813
  -0.1158741 ]
 [ 0.02102338 -0.03948994  0.02134902 ... -0.03727025 -0.07971643
  -0.00752137]
 [ 0.04864246 -0.02335854  0.13749954 ... -0.10765366  0.04497682
  -0.08829252]
 ...
 [-0.06597637 -0.06548307 -0.05338138 ... -0.07340642 -0.11059001
  -0.12669243]
 [-0.09226173  0.04365811  0.06232785 ... -0.05720239 -0.05338946
  -0.16088803]
 [ 0.10310815 -0.06137793  0.05642555 ... -0.15567093 -0.0634543
  -0.04840209]]
(960, 300)


In [14]:
## Training set

## predict
print(X_train_char.shape) #(412059, 960)
y_pred = model.apply(params, X_train_char)
print(y.shape) #(412059, 78)

## score
y_pred_classes = helpers._proba_to_classes(y_pred, "sherlock")

print(y_pred_classes)
print(y_pred_classes.shape)

print(f1_score(y_train, y_pred_classes, average="weighted"))
print(accuracy_score(y_train, y_pred_classes))

(412059, 960)
(412059, 78)
['jockey' 'jockey' 'jockey' ... 'jockey' 'jockey' 'jockey']
(412059,)
0.0008584845270142998
0.02093389538876714


In [15]:
## Validation set

## predict
print(X_val_char.shape) #(412059, 960)
y_val = model.apply(params, X_val_char)
print(y_val.shape) #(412059, 78)

## score
y_val_pred_classes = helpers._proba_to_classes(y_val, "sherlock")

print(y_val_pred_classes)

print(f1_score(y_validation, y_val_pred_classes, average="weighted"))
print(accuracy_score(y_validation, y_val_pred_classes))

(137353, 960)
(137353, 78)
['jockey' 'jockey' 'jockey' ... 'jockey' 'jockey' 'jockey']
0.0008243477687499784
0.020509198925396606


Other feature models

In [22]:
from tensorflow.keras.layers import (
    Input,
    Dense,
    Dropout,
    BatchNormalization,
    concatenate,
)
input_ = np.arange(10).reshape(5, 2)
print(input_)


x1 = tf.keras.layers.Dense(8)(input_)
x2 = tf.keras.layers.Dense(8)(input_)

x1

[[0 1]
 [2 3]
 [4 5]
 [6 7]
 [8 9]]


<tf.Tensor: shape=(5, 8), dtype=float32, numpy=
array([[ 0.07428205, -0.0114342 , -0.43905738, -0.3311604 ,  0.02759314,
         0.47437763, -0.42301235,  0.55930805],
       [ 0.1284771 , -1.1319422 , -2.542329  , -1.7167022 , -0.81129485,
         0.8984709 , -0.36999488,  1.3511837 ],
       [ 0.18267214, -2.25245   , -4.645601  , -3.1022441 , -1.6501828 ,
         1.3225641 , -0.3169775 ,  2.1430593 ],
       [ 0.23686719, -3.3729582 , -6.7488728 , -4.4877863 , -2.489071  ,
         1.7466574 , -0.26396012,  2.9349349 ],
       [ 0.29106224, -4.493466  , -8.852144  , -5.8733277 , -3.3279588 ,
         2.1707506 , -0.2109425 ,  3.7268105 ]], dtype=float32)>

In [31]:
from tensorflow.keras.models import Model, model_from_json

x = Input(shape=(32,))
y = Dense(16, activation='softmax')(x)
model = Model(x, y)

print(x)
print(y)
print(model)

KerasTensor(type_spec=TensorSpec(shape=(None, 32), dtype=tf.float32, name='input_9'), name='input_9', description="created by layer 'input_9'")
KerasTensor(type_spec=TensorSpec(shape=(None, 16), dtype=tf.float32, name=None), name='dense_22/Softmax:0', description="created by layer 'dense_22'")


In [46]:
import sklearn, numpy

T = numpy.array(
  [[1,2,3],    [4,5,6],    [7,8,9]],

  )
print(T.shape)
print(T)
print(T[0])

(3, 3)
[[1 2 3]
 [4 5 6]
 [7 8 9]]
[1 2 3]
